## Requirements

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np

import shapely
from shapely.geometry import Polygon, LinearRing
from scipy.spatial.distance import euclidean

from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score
from sklearn.model_selection import train_test_split

## Constants

In [ ]:
COLORS = ['red', 'green', 'blue']
METRICS = ['std', 'mean']

CHANGE_TYPE_MAP = {'Demolition': 0, 'Road': 1, 'Residential': 2, 'Commercial': 3, 'Industrial': 4,
                   'Mega Projects': 5}
CHANGE_STATUS_MAP = {'Greenland': 1, 'Land Cleared': 2, 'Materials Introduced': 3,
                     'Prior Construction': 4, 'Excavation': 5, 'Construction Started': 6,
                     'Construction Midway': 7, 'Materials Dumped': 8, 'Construction Done': 9,
                     'Operational': 10}

GEOGRAPHY_TYPES = ['Dense Forest', 'Grass Land', 'Sparse Forest', 'Farms', 'River',
                   'Coastal', 'Lakes', 'Barren Land', 'Desert', 'Hills', 'Snow', 'N,A']
URBAN_TYPES = ['Sparse Urban', 'Rural', 'Dense Urban', 'Urban Slum', 'Industrial', 'N,A']

COLUMNS_TO_DROP = ['geography_type', 'urban_type', 'geometry']
DATE_COLUMNS = ['date0', 'date1', 'date2', 'date3', 'date4']

BINARY_FEATURES = ['Dense Forest', 'Grass Land', 'Sparse Forest', 'Farms', 'River',
                   'Coastal', 'Lakes', 'Barren Land', 'Desert', 'Hills', 'Snow',
                   'Sparse Urban', 'Rural', 'Dense Urban', 'Urban Slum', 'Industrial']
CATEGORICAL_FEATURES = ['change_status_date0', 'change_status_date1', 'change_status_date2', 'change_status_date3',
                      'change_status_date4']


CHANGE_STATUS_VALUE_COLUMNS = ['change_status_value_date0', 'change_status_value_date1', 'change_status_value_date2', 'change_status_value_date3', 'change_status_value_date4']
CHANGE_STATUS_COLUMNS = ['change_status_date0', 'change_status_date1', 'change_status_date2', 'change_status_date3', 'change_status_date4']


## Read data

In [ ]:
## Read data
train_df = gpd.read_file('../data/train.geojson', index_col=0)
test_df = gpd.read_file('../data/test.geojson', index_col=0)

In [ ]:
# Create copy of training data
copied_train_df = train_df.copy(deep=True)

## Data treatment

### Mapping strings into integers

In [ ]:
# Maps change_type and change_status values into integers 
copied_train_df['change_type'] = copied_train_df['change_type'].map(CHANGE_TYPE_MAP)
for i in range(5): copied_train_df[f'change_status_date{i}'] = copied_train_df[f'change_status_date{i}'].map(CHANGE_STATUS_MAP)

### One-hot encoding

In [ ]:
# One-hot encoding
for geograph_type in GEOGRAPHY_TYPES:
    copied_train_df["geography_type" + geograph_type] = copied_train_df['geography_type'].apply(lambda x: 1 if geograph_type in x else 0)
for urban_type in URBAN_TYPES:
    copied_train_df["urban_type" + urban_type] = copied_train_df['urban_type'].apply(lambda x: 1 if urban_type in x else 0)


### Sorting of date-related features 

In [ ]:
def sort_date_related_features(row):

    # Sort columns by date
    columns_order = np.argsort(row[DATE_COLUMNS].values)
    new_row = row.copy(deep=True)

    # Update date and change_status order
    for i in range(5):
        new_row[f'date{i}'] = row[f'date{columns_order[i]}']
        new_row[f'change_status_date{i}'] = row[f'change_status_date{columns_order[i]}']

    # Update color metrics order
    for metric in METRICS:
        for color in COLORS:
            for i in range(1, 6):
                new_row[f'img_{color}_{metric}_date{i}'] = row[f'img_{color}_{metric}_date{columns_order[i-1]+1}']

    return new_row

# Fix data type
copied_train_df[DATE_COLUMNS] = copied_train_df[DATE_COLUMNS].apply(lambda x: pd.to_datetime(x, format='%d-%m-%Y', errors='coerce'))

# Sort date related features
copied_train_df = copied_train_df.apply(sort_date_related_features, axis=1)

# Turning date features into float values
copied_train_df[DATE_COLUMNS] = copied_train_df[DATE_COLUMNS].apply(np.float64)
copied_train_df.loc[:, DATE_COLUMNS] = copied_train_df.loc[:, DATE_COLUMNS].applymap(lambda x: np.nan if x < 0 else x)


## Feature engineering

### Geometric feature creation

In [ ]:
def is_paralelogram(polygon):
    it_is = True
    LIMIT = 1e-2
    # Get the outer boundary coordinates
    coords = polygon.exterior.coords[:-1]  # Exclude the closing coordinate
    
    # Check if the polygon has four sides
    num_vertices = len(coords)
    if  num_vertices != 4:  # a paralelogram should have 4 vertices
        it_is = False
        return (it_is, num_vertices, 10*LIMIT, num_vertices*4*LIMIT)

    # Calculate the lengths of the sides
    side_lengths = [euclidean(coords[i],coords[(i+1)%4]) for i in range(4)]  # Calculate the length between each pair of adjacent vertices
    
    # Check if opposite sides have equal length (index 0 and 2, index 1 and 3)
    length_dif = abs(side_lengths[0] - side_lengths[2]) + abs(side_lengths[1] - side_lengths[3])/polygon.length
    if length_dif > LIMIT:
        it_is = False
    
    return (it_is, num_vertices, length_dif, length_dif)

def get_centroid_lat(row):
    center = row['geometry'].centroid.xy[1][0]
    return center

def get_centroid_lon(row):
    center = row['geometry'].centroid.xy[0][0]
    return center


## Create new polygon features

copied_train_df['area'] = copied_train_df['geometry'].area
copied_train_df['length'] = copied_train_df['geometry'].length
copied_train_df['centroid_x'] = copied_train_df['geometry'].centroid.x
copied_train_df['centroid_y'] = copied_train_df['geometry'].centroid.y

bounds = copied_train_df['geometry'].bounds
copied_train_df['angle'] = np.arctan((bounds['maxy']-bounds['miny'])/(bounds['maxx']-bounds['minx']))
copied_train_df['compactness'] = 4 * np.pi * (copied_train_df['area'] / copied_train_df['length']**2)

tmp = copied_train_df['geometry'].apply(is_paralelogram)
copied_train_df['paralelogram'], copied_train_df['num_vertices'], copied_train_df['length_dif1'], copied_train_df['length_dif2'] = zip(*tmp)
copied_train_df['rect_area'] = copied_train_df['geometry'].apply(shapely.minimum_rotated_rectangle, axis=1).area

convex_hull = copied_train_df['geometry'].convex_hull
copied_train_df['dif_convex_prop_area'] = (convex_hull.area - copied_train_df['area'])/ convex_hull.area
copied_train_df['convex'] =  copied_train_df['geometry'].geom_equals(convex_hull)

# Get latitude and longitude
train_df_geo = pd.DataFrame()
train_df_geo['Lat'] = copied_train_df.apply(get_centroid_lat, axis=1)
train_df_geo['Lon'] = copied_train_df.apply(get_centroid_lon, axis=1)

# Use kmeans to cluster latitude and longitude data
kmeans = KMeans(n_clusters=4)
kmeans.fit(train_df_geo)
labels = kmeans.labels_
copied_train_df['geo_cluster'] = labels

### Drop of unecessary columns

In [ ]:
copied_train_df = copied_train_df.drop(COLUMNS_TO_DROP, axis=1)

### Impute missing data

In [ ]:
from sklearn.impute import KNNImputer

# Group the DataFrame by "change_type"
grouped = copied_train_df.groupby("change_type")

# Initialize KNNImputer
imputer = KNNImputer(n_neighbors=5,missing_values=np.nan)  # You can adjust the number of neighbors as needed

# Initialize an empty list to store the imputed DataFrames
imputed_dfs = []

# Iterate over each group
for change_type, group in grouped:
    # Drop the "change_type" column before imputation
    group_features = group.drop(columns=["change_type"])
    
    # Impute NaN values within the group
    imputed_values = imputer.fit_transform(group_features)
    
    # Create a DataFrame with imputed values
    imputed_df = pd.DataFrame(imputed_values, columns=group_features.columns, index=group_features.index)
    
    # Concatenate "change_type" column back to the imputed DataFrame
    imputed_df["change_type"] = change_type
    
    # Append the imputed DataFrame to the list
    imputed_dfs.append(imputed_df)

# Concatenate all imputed DataFrames into a single DataFrame
copied_train_df = pd.concat(imputed_dfs)

# Verify if there are still NaN values after imputation
if copied_train_df.isnull().values.any():
    print("There are still NaN values remaining after imputation.")
    print(copied_train_df[copied_train_df.isnull().any(axis=1)])
else:
    print("All NaN values have been imputed successfully.")

In [ ]:
copied_train_df[CHANGE_STATUS_COLUMNS] = copied_train_df[CHANGE_STATUS_COLUMNS].round().astype('int')

### Compute time and color deltas

In [ ]:
# Compute color delta
for metric in METRICS:
    for color in COLORS:
        for i in range(2, 6):
            delta = copied_train_df[f'img_{color}_{metric}_date{i}'] - copied_train_df[f'img_{color}_{metric}_date{i-1}']
            copied_train_df[f'img_{color}_{metric}_delta{i}'] = delta
        copied_train_df[f'img_{color}_{metric}_delta_total'] = copied_train_df[f'img_{color}_{metric}_date5'] - copied_train_df[f'img_{color}_{metric}_date1']
# Compute time delta
for i in range(1, 5):
    copied_train_df[f'date_delta{i}'] = copied_train_df[f'date{i}'] - copied_train_df[f'date{i-1}']
copied_train_df['date_delta_total'] = copied_train_df[f'date4'] - copied_train_df[f'date1']

### Slopes and Rates computation

In [ ]:
## Standardizing colors mean by the proportion
for i in range(1, 6):
    color_sum = copied_train_df[f'img_blue_mean_date{i}'] + copied_train_df[f'img_green_mean_date{i}'] + copied_train_df[f'img_red_mean_date{i}']
    for color in COLORS:
        copied_train_df[f'img_{color}_mean_prop_date{i}'] = copied_train_df[f'img_{color}_mean_date{i}']/color_sum

In [ ]:
## Create img_{color}_mean_prop_rate
num_samples = copied_train_df.shape[0]
ones = np.ones((num_samples,5,1))

for color in COLORS:
    coef = np.zeros((num_samples))
    COLOR_MEAN_COLUMNS = [f'img_{color}_mean_prop_date{i}' for i in range (1,6)]

    Y = np.array(copied_train_df[COLOR_MEAN_COLUMNS].astype(float))
    X = np.array(copied_train_df[DATE_COLUMNS].apply(np.float64))[:,:,np.newaxis]
    X = np.dstack((ones,X))
    nan_mask = np.isnan(Y) | np.isnan(X[:,:,1])
    X[nan_mask,:] = 0
    Y[nan_mask] = 0

    eye = np.eye(2)*0.0001
    for i in range(num_samples):
        x = X[i].reshape((5,2))
        y = Y[i].reshape((5))
        coef[i] = (np.linalg.inv(eye+x.T@x)@x.T@y)[1]

    copied_train_df[f'img_{color}_mean_prop_rate'] = coef

In [ ]:
## Create img_{color}_std_rate
for color in COLORS:
    coef = np.zeros((num_samples))
    COLOR_STD_COLUMNS = [f'img_{color}_std_date{i}' for i in range (1,6)]

    Y = np.array(copied_train_df[COLOR_STD_COLUMNS].astype(float))
    X = np.array(copied_train_df[DATE_COLUMNS].apply(np.float64))[:,:,np.newaxis]
    X = np.dstack((ones,X))
    nan_mask = np.isnan(Y) | np.isnan(X[:,:,1])
    X[nan_mask,:] = 0
    Y[nan_mask] = 0

    eye = np.eye(2)*0.0001
    for i in range(num_samples):
        x = X[i].reshape((5,2))
        y = Y[i].reshape((5))
        coef[i] = (np.linalg.inv(eye+x.T@x)@x.T@y)[1]

    copied_train_df[f'img_{color}_std_rate'] = coef

In [ ]:
## Create civilization_rate
num_samples = copied_train_df.shape[0]
coef = np.zeros((num_samples))
time_ctt = 1e9*60*90*24
ones = np.ones((num_samples,5,1))

Y = np.array(copied_train_df[CHANGE_STATUS_COLUMNS].astype(float))
Y_nan_mask = np.isnan(Y)
X = np.array(copied_train_df[DATE_COLUMNS].apply(np.float64))[:,:,np.newaxis]/time_ctt
X = np.dstack((ones,X))
X[Y_nan_mask,:] = 0
Y[Y_nan_mask] = 0

eye = np.eye(2)*0.0001
for i in range(num_samples):
    x = X[i].reshape((5,2))
    y = Y[i].reshape((5))
    coef[i] = (np.linalg.inv(eye+x.T@x)@x.T@y)[1]
    #print(y, train_df["change_type"].iloc[i])
copied_train_df["civilizating_rate"] = coef

In [ ]:
ready_to_train_df = copied_train_df.copy(deep=True)

## Model

Our architecture utilizes three models to do the prediction. A brief summary is presented below:
-   We train a first random forest to predict whether change type is in (1, 2, 3) or (4, 5)
-   We train a second random forest to separate classes inside group (4, 5) to 4 or 5
-   We train a third random forest to separate classes inside group (1, 2, 3) to 1 or 2 or 3

### Create groups (1, 2 ,3) and (4, 5)

In [ ]:
ready_to_train_df["industrial_and_megaprojects"] = (ready_to_train_df["change_type"] > 3).replace({True: 1, False: 0})
indus_mega_ready_to_train_df = ready_to_train_df.loc[ready_to_train_df['industrial_and_megaprojects'] == 1]
others_ready_to_train_df = ready_to_train_df.loc[ready_to_train_df['industrial_and_megaprojects'] == 0]

### Undersampling of most common target values

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=142)
X = ready_to_train_df.loc[:, ready_to_train_df.columns != "industrial_and_megaprojects"]
y = ready_to_train_df["industrial_and_megaprojects"]
X_resampled, y_resampled = rus.fit_resample(X, y)

In [ ]:
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=142)
X = ready_to_train_df.loc[:, ready_to_train_df.columns != "industrial_and_megaprojects"]
y = ready_to_train_df["industrial_and_megaprojects"]
X_resampled, y_resampled = cc.fit_resample(X, y)


In [ ]:
train_concated = pd.concat([X_resampled, y_resampled], axis=1)

### Round numerical values to reduce computational burden

In [ ]:
features_to_round = ['change_status_date0',
 'change_status_date1',
 'change_status_date2',
 'change_status_date3',
 'change_status_date4','geography_typeDense Forest',
 'geography_typeGrass Land',
 'geography_typeSparse Forest',
 'geography_typeFarms',
 'geography_typeRiver',
 'geography_typeCoastal',
 'geography_typeLakes',
 'geography_typeBarren Land',
 'geography_typeDesert',
 'geography_typeHills',
 'geography_typeSnow',
 'geography_typeN,A',
 'urban_typeSparse Urban',
 'urban_typeRural',
 'urban_typeDense Urban',
 'urban_typeUrban Slum',
 'urban_typeIndustrial',
 'urban_typeN,A',
]
train_concated[features_to_round] = train_concated[features_to_round].round().astype(int)

### Train first tree

In [ ]:
first_tree_features =  [
'length','centroid_y', 'centroid_x', 'civilizating_rate', 'geography_typeFarms',
'geography_typeCoastal', 'geography_typeLakes', 'urban_typeUrban Slum', 'urban_typeIndustrial',
'date0', 'img_red_mean_prop_rate', 'img_green_mean_prop_rate', 'img_blue_mean_prop_rate',
'area', 'date_delta_total', 'change_status_date4', 'date1', 'compactness', 'img_red_std_rate', 'paralelogram',
'num_vertices', 'length_dif1', 'length_dif2'
]

In [ ]:
# Divide dataset and fit first model

y = ready_to_train_df["industrial_and_megaprojects"]
X = ready_to_train_df[first_tree_features]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True)

clf_first_tree = RandomForestClassifier(n_estimators=50, random_state=42, min_samples_leaf=1)
clf_first_tree.fit(X_train, y_train)

In [ ]:
# Results

y_predicted_train = clf_first_tree.predict(X_train)
y_predicted_real = clf_first_tree.predict(ready_to_train_df[first_tree_features])
y_predicted_test = clf_first_tree.predict(X_test)

print('Train:')
print(precision_score(y_train, y_predicted_train, average="micro"))
print(f1_score(y_train, y_predicted_train, average="macro"))
print('Real:')
print(precision_score(ready_to_train_df["industrial_and_megaprojects"], y_predicted_real, average="micro"))
print(f1_score(ready_to_train_df["industrial_and_megaprojects"], y_predicted_real, average="macro"))
print('Test:')
print(precision_score(y_test, y_predicted_test, average="micro"))
print(f1_score(y_test, y_predicted_test, average="macro"))


### Train second tree

In [ ]:
second_tree_features =  [
'length','centroid_y', 'centroid_x', 'civilizating_rate', 'geography_typeFarms',
'geography_typeCoastal', 'geography_typeLakes', 'urban_typeUrban Slum', 'urban_typeIndustrial',
'date0', 'img_red_mean_prop_rate', 'img_green_mean_prop_rate', 'img_blue_mean_prop_rate',
'area', 'date_delta_total', 'change_status_date4', 'date1', 'compactness', 'img_red_std_rate', 'paralelogram',
'num_vertices', 'length_dif1', 'length_dif2'
]

In [ ]:
# Divide dataset and fit second model 
y = indus_mega_ready_to_train_df["change_type"]
X = indus_mega_ready_to_train_df[second_tree_features]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True)

clf_second_tree = RandomForestClassifier(n_estimators=50, random_state=42, min_samples_leaf=1)
clf_second_tree.fit(X_train, y_train)

In [ ]:
# Results

y_predicted_train = clf_second_tree.predict(X_train)
y_predicted_test = clf_second_tree.predict(X_test)

print('Train:')
print(precision_score(y_train, y_predicted_train, average="micro"))
print(f1_score(y_train, y_predicted_train, average="macro"))
print('Test:')
print(precision_score(y_test, y_predicted_test, average="micro"))
print(f1_score(y_test, y_predicted_test, average="macro"))



### Third tree

In [ ]:
third_tree_features =  [
'length','centroid_y', 'centroid_x', 'civilizating_rate', 'geography_typeFarms',
'geography_typeCoastal', 'geography_typeLakes', 'urban_typeUrban Slum', 'urban_typeIndustrial',
'date0', 'img_red_mean_prop_rate', 'img_green_mean_prop_rate', 'img_blue_mean_prop_rate',
'area', 'date_delta_total', 'change_status_date4', 'date1', 'compactness', 'img_red_std_rate', 'paralelogram',
'num_vertices', 'length_dif1', 'length_dif2'
]

In [ ]:
# Divide dataset and fit third model 
y = others_ready_to_train_df["change_type"]
X = others_ready_to_train_df[third_tree_features]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True)


clf_third_tree = RandomForestClassifier(n_estimators=100, random_state=42, min_samples_leaf=1)
clf_third_tree.fit(X_train, y_train)

In [ ]:
# Results

y_predicted_train = clf_third_tree.predict(X_train)
y_predicted_test = clf_third_tree.predict(X_test)

print('Train:')
print(precision_score(y_train, y_predicted_train, average="micro"))
print(f1_score(y_train, y_predicted_train, average="macro"))
print('Test:')
print(precision_score(y_test, y_predicted_test, average="micro"))
print(f1_score(y_test, y_predicted_test, average="macro"))

## Kaggle submission

### Apply same transformations made in train_df to test_df

In [ ]:
# Create copy of test data
copied_test_df = test_df.copy(deep=True)

In [ ]:
# Maps change_type and change_status values into integers 
for i in range(5): copied_test_df[f'change_status_date{i}'] = copied_test_df[f'change_status_date{i}'].map(CHANGE_STATUS_MAP)

In [ ]:
# One-hot encoding
for geograph_type in GEOGRAPHY_TYPES:
    copied_test_df["geography_type" + geograph_type] = copied_test_df['geography_type'].apply(lambda x: 1 if geograph_type in x else 0)
for urban_type in URBAN_TYPES:
    copied_test_df["urban_type" + urban_type] = copied_test_df['urban_type'].apply(lambda x: 1 if urban_type in x else 0)


In [ ]:
# Fix data type
copied_test_df[DATE_COLUMNS] = copied_test_df[DATE_COLUMNS].apply(lambda x: pd.to_datetime(x, format='%d-%m-%Y', errors='coerce'))

# Sort date related features
copied_test_df = copied_test_df.apply(sort_date_related_features, axis=1)

# Turning date features into float values
copied_test_df[DATE_COLUMNS] = copied_test_df[DATE_COLUMNS].apply(np.float64)
copied_test_df.loc[:, DATE_COLUMNS] = copied_test_df.loc[:, DATE_COLUMNS].applymap(lambda x: np.nan if x < 0 else x)


In [ ]:
## Create new polygon features

copied_test_df['area'] = copied_test_df['geometry'].area
copied_test_df['length'] = copied_test_df['geometry'].length
copied_test_df['centroid_x'] = copied_test_df['geometry'].centroid.x
copied_test_df['centroid_y'] = copied_test_df['geometry'].centroid.y

bounds = copied_test_df['geometry'].bounds
copied_test_df['angle'] = np.arctan((bounds['maxy']-bounds['miny'])/(bounds['maxx']-bounds['minx']))
copied_test_df['compactness'] = 4 * np.pi * (copied_test_df['area'] / copied_test_df['length']**2)

tmp = copied_test_df['geometry'].apply(is_paralelogram)
copied_test_df['paralelogram'], copied_test_df['num_vertices'], copied_test_df['length_dif1'], copied_test_df['length_dif2'] = zip(*tmp)
copied_test_df['rect_area'] = copied_test_df['geometry'].apply(shapely.minimum_rotated_rectangle, axis=1).area

convex_hull = copied_test_df['geometry'].convex_hull
copied_test_df['dif_convex_prop_area'] = (convex_hull.area - copied_test_df['area'])/ convex_hull.area
copied_test_df['convex'] =  copied_test_df['geometry'].geom_equals(convex_hull)

# Get latitude and longitude
test_df_geo = pd.DataFrame()
test_df_geo['Lat'] = copied_test_df.apply(get_centroid_lat, axis=1)
test_df_geo['Lon'] = copied_test_df.apply(get_centroid_lon, axis=1)

# Use kmeans to cluster latitude and longitude data
kmeans = KMeans(n_clusters=4)
kmeans.fit(test_df_geo)
labels = kmeans.labels_
copied_test_df['geo_cluster'] = labels

In [ ]:
# Drop unecessary columns
copied_test_df = copied_test_df.drop(COLUMNS_TO_DROP, axis=1)

In [ ]:
### Impute missing data ###

# Group the DataFrame by "change_type"
grouped = copied_test_df.groupby("change_type")

# Initialize KNNImputer
imputer = KNNImputer(n_neighbors=5,missing_values=np.nan)  # You can adjust the number of neighbors as needed

# Initialize an empty list to store the imputed DataFrames
imputed_dfs = []

# Iterate over each group
for change_type, group in grouped:
    # Drop the "change_type" column before imputation
    group_features = group.drop(columns=["change_type"])
    
    # Impute NaN values within the group
    imputed_values = imputer.fit_transform(group_features)
    
    # Create a DataFrame with imputed values
    imputed_df = pd.DataFrame(imputed_values, columns=group_features.columns, index=group_features.index)
    
    # Concatenate "change_type" column back to the imputed DataFrame
    imputed_df["change_type"] = change_type
    
    # Append the imputed DataFrame to the list
    imputed_dfs.append(imputed_df)

# Concatenate all imputed DataFrames into a single DataFrame
copied_test_df = pd.concat(imputed_dfs)

# Verify if there are still NaN values after imputation
if copied_test_df.isnull().values.any():
    print("There are still NaN values remaining after imputation.")
    print(copied_test_df[copied_test_df.isnull().any(axis=1)])
else:
    print("All NaN values have been imputed successfully.")

In [ ]:
copied_test_df[CHANGE_STATUS_COLUMNS] = copied_test_df[CHANGE_STATUS_COLUMNS].round().astype('int')

In [ ]:
# Compute color delta
for metric in METRICS:
    for color in COLORS:
        for i in range(2, 6):
            delta = copied_test_df[f'img_{color}_{metric}_date{i}'] - copied_test_df[f'img_{color}_{metric}_date{i-1}']
            copied_test_df[f'img_{color}_{metric}_delta{i}'] = delta
        copied_test_df[f'img_{color}_{metric}_delta_total'] = copied_test_df[f'img_{color}_{metric}_date5'] - copied_test_df[f'img_{color}_{metric}_date1']
# Compute time delta
for i in range(1, 5):
    copied_test_df[f'date_delta{i}'] = copied_test_df[f'date{i}'] - copied_test_df[f'date{i-1}']
copied_test_df['date_delta_total'] = copied_test_df[f'date4'] - copied_test_df[f'date1']

In [ ]:
## Standardizing colors mean by the proportion
for i in range(1, 6):
    color_sum = copied_test_df[f'img_blue_mean_date{i}'] + copied_test_df[f'img_green_mean_date{i}'] + copied_test_df[f'img_red_mean_date{i}']
    for color in COLORS:
        copied_test_df[f'img_{color}_mean_prop_date{i}'] = copied_test_df[f'img_{color}_mean_date{i}']/color_sum

In [ ]:
## Create img_{color}_mean_prop_rate
num_samples = copied_test_df.shape[0]
ones = np.ones((num_samples,5,1))

for color in COLORS:
    coef = np.zeros((num_samples))
    COLOR_MEAN_COLUMNS = [f'img_{color}_mean_prop_date{i}' for i in range (1,6)]

    Y = np.array(copied_test_df[COLOR_MEAN_COLUMNS].astype(float))
    X = np.array(copied_test_df[DATE_COLUMNS].apply(np.float64))[:,:,np.newaxis]
    X = np.dstack((ones,X))
    nan_mask = np.isnan(Y) | np.isnan(X[:,:,1])
    X[nan_mask,:] = 0
    Y[nan_mask] = 0

    eye = np.eye(2)*0.0001
    for i in range(num_samples):
        x = X[i].reshape((5,2))
        y = Y[i].reshape((5))
        coef[i] = (np.linalg.inv(eye+x.T@x)@x.T@y)[1]

    copied_test_df[f'img_{color}_mean_prop_rate'] = coef

In [ ]:
## Create img_{color}_std_rate
for color in COLORS:
    coef = np.zeros((num_samples))
    COLOR_STD_COLUMNS = [f'img_{color}_std_date{i}' for i in range (1,6)]

    Y = np.array(copied_test_df[COLOR_STD_COLUMNS].astype(float))
    X = np.array(copied_test_df[DATE_COLUMNS].apply(np.float64))[:,:,np.newaxis]
    X = np.dstack((ones,X))
    nan_mask = np.isnan(Y) | np.isnan(X[:,:,1])
    X[nan_mask,:] = 0
    Y[nan_mask] = 0

    eye = np.eye(2)*0.0001
    for i in range(num_samples):
        x = X[i].reshape((5,2))
        y = Y[i].reshape((5))
        coef[i] = (np.linalg.inv(eye+x.T@x)@x.T@y)[1]

    copied_test_df[f'img_{color}_std_rate'] = coef

In [ ]:
## Create civilization_rate
num_samples = copied_test_df.shape[0]
coef = np.zeros((num_samples))
time_ctt = 1e9*60*90*24
ones = np.ones((num_samples,5,1))

Y = np.array(copied_test_df[CHANGE_STATUS_COLUMNS].astype(float))
Y_nan_mask = np.isnan(Y)
X = np.array(copied_test_df[DATE_COLUMNS].apply(np.float64))[:,:,np.newaxis]/time_ctt
X = np.dstack((ones,X))
X[Y_nan_mask,:] = 0
Y[Y_nan_mask] = 0

eye = np.eye(2)*0.0001
for i in range(num_samples):
    x = X[i].reshape((5,2))
    y = Y[i].reshape((5))
    coef[i] = (np.linalg.inv(eye+x.T@x)@x.T@y)[1]
copied_test_df["civilizating_rate"] = coef

In [ ]:
ready_to_test_df = copied_test_df.copy(deep=True)

### Run predictions on test

In [ ]:
y_test_predicted1 = clf_first_tree.predict(ready_to_test_df[first_tree_features])
y_test_predicted2 = clf_second_tree.predict(ready_to_test_df[second_tree_features])
y_test_predicted3 = clf_third_tree.predict(ready_to_test_df[third_tree_features])

### Format and save results

In [ ]:
def final_result(row):
   if row[0] == 0:
      return row[2]
   if row[0] == 1:
      return row[1]

options = pd.DataFrame(np.array([y_test_predicted1, y_test_predicted2, y_test_predicted3]).T)   
options["change_type"] = options.apply(final_result, axis=1)

In [ ]:
options["change_type"].to_csv("to_submit.csv")